In [ ]:
pip install matplotlib

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Choose a random index
index = np.random.randint(0, len(x_train))

# Get the image and its label
image = x_train[index]
label = y_train[index][0]


# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Display the image
plt.figure(figsize=(4,4))
plt.imshow(image)
plt.title(f"Class: {class_names[label]}")
plt.axis('off')
plt.show()

print(f"Image shape: {image.shape}")
print(f"Class: {class_names[label]}")

: 

In [ ]:
#output multiple images from the dataset
for i in range(10):
    plt.figure(figsize=(4,4))
    plt.imshow(x_train[i])
    plt.title(f"Class: {class_names[y_train[i][0]]}")
    plt.axis('off')
    plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


In [ ]:
model.save('cifar10_model.h5')


# 2. Convert the Model to TensorFlow Lite

In [ ]:
model = tf.keras.models.load_model('cifar10_model.h5')


In [ ]:
def representative_dataset():
    for i in range(100):
        yield [x_train[i:i+1]]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()


In [ ]:
with open('cifar10_model_quant.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
#Objective: Convert the TensorFlow Lite model into a C array to include it in the ESP-IDF project.

And then put the following in the commandline: 

xxd -i cifar10_model_quant.tflite > model.h
